In [1]:
import os
import shutil
import logging
import tkinter as tk
from tkinter import filedialog, messagebox

# Initialize the dictionary to store the original locations of the files
original_locations = {}
sorted_folders = []  # List to keep track of folders created during sorting

def get_file_type(file_name):
    # Function to get the file type based on the extension
    extension = os.path.splitext(file_name)[1].lower()
    if extension == '.pdf':
        return 'pdf files'
    elif extension == '.pptx':
        return 'pptx files'
    elif extension == '.exe':
        return 'exe files'
    elif extension in ['.jpg', '.jpeg', '.png']:
        return 'image files'
    else:
        # Handling files with no extension as "other files"
        return 'other files'

def confirm(prompt="Do you want to proceed? (y/n): "):
    return messagebox.askyesno("Confirmation", prompt)

def move_files(source, destination):
    # Move files from the source directory to the destination directory
    for file in os.listdir(source):
        shutil.move(os.path.join(source, file), os.path.join(destination, file))
        logging.info(f"Moved '{file}' to '{destination}'.")

def remove_empty_folders(path):
    # Function to remove empty folders recursively
    if not os.path.exists(path):
        return

    for folder_name in os.listdir(path):
        folder_path = os.path.join(path, folder_name)
        if os.path.isdir(folder_path):
            remove_empty_folders(folder_path)

    # After the recursive call, check if the current directory is empty
    if not os.listdir(path):
        os.rmdir(path)

def sort_files():
    source_folder = filedialog.askdirectory(title="Select Source Folder")
    if source_folder:
        if confirm("Do you want to proceed with file sorting?"):
            # Record the original locations of the files before sorting
            for file in os.listdir(source_folder):
                original_locations[file] = source_folder  # Store the current location

            # Sorting process
            for file in os.listdir(source_folder):
                file_type = get_file_type(file)
                destination_folder = os.path.join(source_folder, file_type)
                if not os.path.exists(destination_folder):
                    os.makedirs(destination_folder)
                    sorted_folders.append(destination_folder)

                shutil.move(os.path.join(source_folder, file), os.path.join(destination_folder, file))
                logging.info(f"Moved '{file}' to '{destination_folder}'.")

            messagebox.showinfo("File Sorting", "File sorting completed successfully.")
        else:
            messagebox.showinfo("File Sorting", "File sorting was canceled.")

            
def undo_sorting_single_folder(folder):
    if folder in sorted_folders:
        # Undo process for a single folder
        for file in os.listdir(folder):
            original_path = original_locations.get(file)
            if original_path is not None and os.path.exists(os.path.join(folder, file)):
                shutil.move(os.path.join(folder, file), os.path.join(original_path, file))
                logging.info(f"Restored '{file}' to '{original_path}'.")
            else:
                logging.warning(f"File '{file}' not found in the original location during undo.")

        remove_empty_folders(folder)
#         messagebox.showinfo("Undo Sorting", "Undo completed successfully.")
    else:
        messagebox.showerror("Error", "This folder was not created during the sorting process.")

def undo_sorting_all_folders():
    # Undo process for all folders created during sorting
    for folder in sorted_folders:
        undo_sorting_single_folder(folder)
    messagebox.showinfo("Undo Sorting", "Undo completed successfully.")

# Configure the logging settings
logging.basicConfig(
    filename='file_sorting.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Create the main application window
app = tk.Tk()
app.title("File Sorting Application")

# Add buttons for sorting and undoing
sort_button = tk.Button(app, text="Sort Files", command=sort_files)
sort_button.pack(pady=10)

undo_single_button = tk.Button(app, text="Undo Sorting (Single Folder)", command=lambda: undo_sorting_single_folder(filedialog.askdirectory(title="Select Folder to Undo Sorting")))
undo_single_button.pack(pady=5)

undo_all_button = tk.Button(app, text="Undo Sorting (All Folders)", command=undo_sorting_all_folders)
undo_all_button.pack(pady=5)

# Run the main event loop
app.mainloop()

# Save and close the log file
logging.shutdown()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\DELL\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\DELL\AppData\Local\Temp/ipykernel_20060/3074870935.py", line 92, in undo_sorting_all_folders
    undo_sorting_single_folder(folder)
  File "C:\Users\DELL\AppData\Local\Temp/ipykernel_20060/3074870935.py", line 76, in undo_sorting_single_folder
    for file in os.listdir(folder):
FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:/Users/DELL/OneDrive/Desktop/File sort\\other files'
